In [7]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [8]:
engine = create_engine('postgresql+pychopg2://localhost:5432/sql-challenge')
connection = engine.connect()

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgresql.pychopg2